In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'

import re
import requests
from bs4 import BeautifulSoup

import time
from pathlib import Path

# GetDate
from datetime import datetime #,  date

import glob
import os.path

# Desativacao de warning
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [5]:
#Reimportando arquivo - caso execucao tenha sido pausada
#Identificacao de arquivo mais recente
folder_path = r'./ARQ/'
file_type = r'/*csv'
files = glob.glob(folder_path + file_type)
max_file = max(files, key=os.path.getctime)

path_to_file = max_file.replace('\\','/')

#Em caso de definicao manual usar esta linha
path_to_file = './ARQ/Listagem_Final.csv'

listagem = pd.read_csv(path_to_file,
                       usecols= ['Projeto','Ano','Tipo','Ementa','Situacao','Autor','Codigo_Tipo'
                                 ,'Id_Projeto_Camara','Data_Projeto','Tramitacao_Encerrada','Aprovado'])

#Substitundo valores
listagem['Aprovado'] = listagem['Aprovado'].fillna('')
listagem['Tramitacao_Encerrada'] = listagem['Tramitacao_Encerrada'].fillna('')
listagem['Data_Projeto'] = listagem['Data_Projeto'].fillna('')
listagem['Codigo_Tipo'] = listagem['Codigo_Tipo'].fillna('')

listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].fillna(0)
listagem['Id_Projeto_Camara'] = listagem['Id_Projeto_Camara'].astype(int)

In [6]:
listagem

,Projeto,Ano,Tipo,Ementa,Situacao,Autor,Codigo_Tipo,Id_Projeto_Camara,Data_Projeto,Tramitacao_Encerrada,Aprovado
0,40520,2013,Requerimento,"Solicita urbanização na rua Antônio Celeste, n...",Arquivada,Oliveira Tresse,REQ,44205,11/07/2013,SIM,SIM
1,4028,2013,Requerimento,Solicita limpeza nas bocas de lobo no bairro C...,Arquivada,Júlio Gasparette,REQ,44107,09/07/2013,SIM,SIM
2,36840,2013,Requerimento,solicita recapemamento asfáltico na rua Orland...,Em Tramitação,Luiz Otávio Fernandes Coelho - Pardal,REQ,43978,31/12/2000,NAO,Informacao nao encontrada
3,891,2013,Requerimento,Solicita o asfaltamento de toidas as ruas da C...,Arquivada,Cido,REQ,31106,05/02/2013,SIM,SIM
4,8890,2013,Requerimento,Solicita o asfaltamento de todas as ruas da co...,Arquivada,Cido,REQ,27368,05/02/2013,SIM,SIM
...,...,...,...,...,...,...,...,...,...,...,...
74093,1,2022,Emenda,"Substitui os artigos 2º e 4ª, do Projeto de Le...",Aprovado,Nilton Militão,EMENDA,-1131,,,
74094,1,2022,Emenda,Acrescenta o art.6º ao Projeto de Lei oriundo ...,Aguardando leitura em plenário,"André Luiz, Dr. Antônio Aguiar",EMENDA,-1132,,,
74095,1,2022,Emenda,"Substitui o caput do artigo 1º, o inciso I do ...",Aprovado,"Luiz Otávio Fernandes Coelho - Pardal, Cida Ol...",EMENDA,-1133,,,
74096,1,2022,Projeto Substitutivo,"“Altera dispositivos da Lei Nº 10.777, de 15 d...",Aprovado,"Zé Márcio, Bejani Júnior, Sargento Mello Casal...",PSUB,121457,13/12/2022,SIM,SIM


In [7]:
# Exportacao da listagem de projetos para arquivo csv
path_to_file = './ARQ/Listagem_Projetos_2012-2022.csv'
path = Path(path_to_file)

exp_columns = ['Projeto', 'Ano', 'Tipo', 'Ementa', 'Situacao', 'Codigo_Tipo'
               , 'Id_Projeto_Camara', 'Data_Projeto','Tramitacao_Encerrada','Aprovado']
              #, 'Autor'] 
    
if path.is_file():
    exp_header = False
else:    
    exp_header = True

listagem.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

In [8]:
# Logica de identificador de autor (Id_Autor) e identificacao de autoria, separando casos de 'multiautoria'
autoria_df_estrutura = {'nome_autor': [''],'id_projeto_camara': ['']}
autoria_df = pd.DataFrame(columns=['nome_autor','id_projeto_camara'])
autores = ['']
aut = ['']

for i in listagem.index:
    aut = listagem['Autor'][i].split(', ')
    
    
    for item in aut:
        autoria = pd.DataFrame({'nome_autor': item, 
                      'id_projeto_camara': listagem['Id_Projeto_Camara'][i]}, index=[0]) 
        autoria_df = autoria_df.append(autoria, ignore_index=True, verify_integrity=False, sort=None) 

# Exportacao da listagem de autorias para arquivo csv
path_to_file = './ARQ/Listagem_Autorias_2012-2022.csv'
path = Path(path_to_file)
exp_columns = ["id_projeto_camara", "nome_autor"]

if path.is_file():
    exp_header = False
else:    
    exp_header = True

autoria_df.to_csv(path_to_file, mode='a', encoding = 'utf-8', index=False, header=exp_header, columns=exp_columns) 

In [9]:
autoria_df

,nome_autor,id_projeto_camara
0,Oliveira Tresse,44205
1,Júlio Gasparette,44107
2,Luiz Otávio Fernandes Coelho - Pardal,43978
3,Cido,31106
4,Cido,27368
...,...,...
79388,Juraci Scheffer,121457
79389,Protetora Kátia Franco,121457
79390,Laiz Perrut,121457
79391,Marlon Siqueira,121457
